In [ ]:
%load ./hello-world.ipynb

In [11]:
import dev.langchain4j.agent.tool.Tool;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.service.SystemMessage;
import dev.langchain4j.model.chat.request.ResponseFormatType;

import java.time.ZoneId;
import java.time.LocalDateTime;
import java.time.format.DateTimeFormatter;

In [ ]:
interface TimeKeeper {
    @SystemMessage("指定したタイムゾーンの現在の時刻を何時何分の形式で答えてください")
    String ask(String question);
}

class Clock {
    @Tool
    public LocalDateTime getCurrentTimeInTimeZone(String timeZoneId) {
        System.out.println("タイムゾーン: " + timeZoneId);
        return LocalDateTime.now(ZoneId.of(timeZoneId));
    }
}

TimeKeeper timeKeeper = AiServices.builder(TimeKeeper.class)
    .chatLanguageModel(model)
    .tools(new Clock())
    .build();

String answer = timeKeeper.ask("ロンドンの時間で今何時？");
System.out.println(answer);  // 例: "現在の時刻は16:58です"

In [ ]:
interface MathGenius {    
    String ask(String question);
}

class Calculator {
    @Tool
    public double add(int a, int b) {
        return a + b;
    }
    @Tool
    public double squareRoot(double x) {
        return Math.sqrt(x);
    }
}

MathGenius mathGenius = AiServices.builder(MathGenius.class)
    .chatLanguageModel(model)
    .tools(new Calculator())
    .build();

String answer = mathGenius.ask("475695037565 の平方根は？");
answer

In [18]:
OpenAiChatModel strictModel = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName("gpt-4o-mini")
    .logRequests(true)
    .logResponses(true)
    .responseFormat("json_schema") // JSONスキーマ出力を強制する
    .strictJsonSchema(true) // JSONスキーマ出力を強制する
    .build();

In [ ]:
enum Operation {
    ADD, SQUARE_ROOT;
}
record Calculation(Operation operation, List<Double> numbers) {}

interface TextToCalculation {
    Calculation classify(String text);
}

TextToCalculation classifier = AiServices.create(TextToCalculation.class, strictModel);
// String question = "475695037565 の平方根は？";
String question = "111と222と333の和は？";
Calculation calc = classifier.classify(question);
double answer;
if (calc.operation() == Operation.SQUARE_ROOT) {
    answer = Math.sqrt(calc.numbers().get(0));    
} else if (calc.operation() == Operation.ADD) {
    answer = calc.numbers().stream().reduce(0.0, Double::sum);
}
answer